In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('horse_data.csv', names = ['Fact of surgery', 'Age', 'Hospital Number', 'Temperature', 'Pulse', 'Respiratory Rate', 'Temperature of Extremities', 'Peripheral Pulse', 'Mucous Membranes', 'Capillary Refill Time','Pain', 'Peristalsis', 'Abdominal Distension', 'Nasogastric Tube', 'Nasogastric Reflux', 'Nasogastric Reflux PH', 'Rectal Examination', 'Abdomen', 'Packed Cell Volume', 'Total Protein', 'Abdominocentesis Appearance', 'Abdomcentesis Total Protein', 'Outcome', 'Surgical Lesion', 'Type of Lesion', 'Type of Lesion_1', 'Type of Lesion_2', 'cp_data'], sep = '\,', header=0, engine='python')
df.head(5)

,Fact of surgery,Age,Hospital Number,Temperature,Pulse,Respiratory Rate,Temperature of Extremities,Peripheral Pulse,Mucous Membranes,Capillary Refill Time,...,Packed Cell Volume,Total Protein,Abdominocentesis Appearance,Abdomcentesis Total Protein,Outcome,Surgical Lesion,Type of Lesion,Type of Lesion_1,Type of Lesion_2,cp_data
0,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
2,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
3,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2
4,2,1,528355,?,?,?,2,1,3,1,...,?,?,?,?,1,2,0,0,0,2


In [3]:
#Отберем колонки для анализа
data = df[['Hospital Number', 'Fact of surgery', 'Temperature', 'Pulse', 'Abdominal Distension', 'Packed Cell Volume','Pain', 'Outcome']].reset_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   index                 299 non-null    int64 
 1   Hospital Number       299 non-null    int64 
 2   Fact of surgery       299 non-null    object
 3   Temperature           299 non-null    object
 4   Pulse                 299 non-null    object
 5   Abdominal Distension  299 non-null    object
 6   Packed Cell Volume    299 non-null    object
 7   Pain                  299 non-null    object
 8   Outcome               299 non-null    object
dtypes: int64(2), object(7)
memory usage: 21.1+ KB


In [4]:
data = data.replace('?', np.nan)
data = data.apply(pd.to_numeric)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 299 non-null    int64  
 1   Hospital Number       299 non-null    int64  
 2   Fact of surgery       298 non-null    float64
 3   Temperature           239 non-null    float64
 4   Pulse                 275 non-null    float64
 5   Abdominal Distension  243 non-null    float64
 6   Packed Cell Volume    270 non-null    float64
 7   Pain                  244 non-null    float64
 8   Outcome               298 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 21.1 KB


In [5]:
data.describe()

,index,Hospital Number,Fact of surgery,Temperature,Pulse,Abdominal Distension,Packed Cell Volume,Pain,Outcome
count,299.000000,2.990000e+02,298.000000,239.000000,275.000000,243.000000,270.000000,244.000000,298.000000
mean,149.000000,1.087748e+06,1.395973,38.166527,71.934545,2.259259,46.300000,2.942623,1.550336
std,86.458082,1.532026e+06,0.489881,0.733508,28.680522,1.061472,10.438384,1.303993,0.737967
min,0.000000,5.184760e+05,1.000000,35.400000,30.000000,1.000000,23.000000,1.000000,1.000000
25%,74.500000,5.289040e+05,1.000000,37.800000,48.000000,1.000000,38.000000,2.000000,1.000000
50%,149.000000,5.303100e+05,1.000000,38.200000,64.000000,2.000000,45.000000,3.000000,1.000000
75%,223.500000,5.347360e+05,2.000000,38.500000,88.000000,3.000000,52.000000,4.000000,2.000000
max,298.000000,5.305629e+06,2.000000,40.800000,184.000000,4.000000,75.000000,5.000000,3.000000


In [6]:
#выбросы по температуре

def ejection(row):
    q1 = data['Temperature'].quantile(0.25)
    q3 = data['Temperature'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Temperature'] > upper_bound or row['Temperature'] < lower_bound:
        return row['Temperature']

In [7]:
"""Возможный разброс температуры для лошадей от 35 до 42 град Цельсия, где 40 и выше соответствует лихорадке при воспалительном процессе, 
а 36 и ниже - шоковому состоянию. Все выбросы находятся в рамках допустимых значений."""
data_1 = pd.DataFrame()
data_1['Temperature ejection'] = data.apply(ejection, axis=1)
data_1.dropna()



,Temperature ejection
19,39.9
43,35.4
53,40.3
74,39.7
79,36.4
90,40.3
98,39.6
117,36.5
140,36.0
237,36.1


In [8]:
def ejection_1(row):
    q1 = data['Pulse'].quantile(0.25)
    q3 = data['Pulse'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Pulse'] > upper_bound or row['Pulse'] < lower_bound:
        return row['Pulse']

In [9]:
data_1 = pd.DataFrame()
data_1['Pulse ejection'] = data.apply(ejection_1, axis=1)
data_1.dropna()
#Физиологическая норма для лошадей от 20 до 240 ударов. Все выбросы имеют возможные значения

,Pulse ejection
2,164.0
40,150.0
54,160.0
254,184.0
274,150.0


In [10]:
def ejection_3(row):
    q1 = data['Packed Cell Volume'].quantile(0.25)
    q3 = data['Packed Cell Volume'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Packed Cell Volume'] > upper_bound or row['Packed Cell Volume'] < lower_bound:
        return row['Packed Cell Volume']

In [11]:
data_1 = pd.DataFrame()
data_1['PCV ejection'] = data.apply(ejection_3, axis=1)
data_1.dropna()
#максимальный гематокрит - 80%, повышенные значения могут быть вызваны обезвоживанием организма, осложнением после операции и тд

,PCV ejection
3,74.0
143,75.0
150,75.0


In [12]:
for col in data:
    pct_missing = data[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

index - 0.0%
Hospital Number - 0.0%
Fact of surgery - 0.3%
Temperature - 20.1%
Pulse - 8.0%
Abdominal Distension - 18.7%
Packed Cell Volume - 9.7%
Pain - 18.4%
Outcome - 0.3%


In [13]:
#найдем пропуск по факту операции
data[data['Fact of surgery'].isnull()]

,index,Hospital Number,Fact of surgery,Temperature,Pulse,Abdominal Distension,Packed Cell Volume,Pain,Outcome
131,131,534572,NaN,38.0,48.0,1.0,43.0,4.0,NaN


In [14]:
# так как значения температуры, пульсы и гематокрита находятся в рамках показаний для здоровой лошади, вздутие отсутствовало, предположим, что показаний к операции не было. Внесем значение вручную
data['Fact of surgery'].fillna(2, inplace=True)

In [15]:
#пропущенное значение исхода заполним исходя из группировок по боли и операции - чем выше эти показания, тем в среднем хуже исход

data.loc[data['Outcome'].isnull(), 'Outcome'] = data.groupby(['Fact of surgery','Pain'])['Outcome'].transform('median')

In [16]:
#из описания данных можно сделать вывод, что чем сильнее вздутие, тем вероятнее операция. Заполним пропуски по столбцу вздутия на основе проведенных операций
data.loc[data['Abdominal Distension'].isnull(), 'Abdominal Distension'] = data.groupby(['Fact of surgery'])['Abdominal Distension'].transform('median')

In [17]:
#пропуски в графе боль заполняем на основании вздутия. Чем оно больше, тем больше боли должно испытывать животное
data.loc[data['Pain'].isnull(), 'Pain'] = data.groupby(['Abdominal Distension'])['Pain'].transform('median')

In [18]:
#показатели температуры, пульса и гематокрита заполним медианами соответствующих значений
data['Temperature'].fillna(data['Temperature'].median(), inplace=True)

In [19]:
data['Pulse'].fillna(data['Pulse'].median(), inplace=True)

In [20]:
data['Packed Cell Volume'].fillna(data['Packed Cell Volume'].median(), inplace=True)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 299 non-null    int64  
 1   Hospital Number       299 non-null    int64  
 2   Fact of surgery       299 non-null    float64
 3   Temperature           299 non-null    float64
 4   Pulse                 299 non-null    float64
 5   Abdominal Distension  299 non-null    float64
 6   Packed Cell Volume    299 non-null    float64
 7   Pain                  299 non-null    float64
 8   Outcome               299 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 21.1 KB
